In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../datasets/non-clean_diabetes.csv')
df.head(5)

,col1,col2,col3,col4,col5,col6,col7,col8,y
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
col_names = [
    'No. of Times Pregnant',
    'Plasma Glucose Concentration',
    'Diastolic blood pressure',
    'Triceps skinfold thickness',
    'Serum insulin',
    'Body mass index',
    'Pedigree function',
    'Age',
    'Class label'
]

df.columns = col_names
df.head(5)

,No. of Times Pregnant,Plasma Glucose Concentration,Diastolic blood pressure,Triceps skinfold thickness,Serum insulin,Body mass index,Pedigree function,Age,Class label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
X = df.drop(columns=['Class label']).values
y = df['Class label'].values

In [11]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=0, strategy='mean')
imp = imp.fit(X)
X = imp.transform(X)
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [12]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
scaled = scalar.fit_transform(X)
X = scaled

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=1)

In [14]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
acc = round(accuracy_score(y_test, y_pred), 2)
print('Acc:', acc)

Acc: 0.67


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[76 24]
 [27 27]]
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       100
           1       0.53      0.50      0.51        54

    accuracy                           0.67       154
   macro avg       0.63      0.63      0.63       154
weighted avg       0.66      0.67      0.67       154



In [18]:
predictions = pd.DataFrame({
        'Actual': y_test, 
        'Predicted': y_pred
    }
)

predictions

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
149,0,1
150,0,1
151,0,0
152,1,0


In [22]:
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
export_graphviz(classifier, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

InvocationException: GraphViz's executables not found